<a href="https://colab.research.google.com/github/caiocesarcosta/AndroidTVappTutorial/blob/master/detection_transfer_learning_yolo_porject4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import random
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import shutil  # Importa a biblioteca shutil para remover diretórios recursivamente
import yaml
import tensorflow as tf
from google.colab import drive  # Importa a biblioteca para montagem do Google Drive

# Monta o Google Drive
drive.mount('/content/drive')

# Configurações
num_images_to_download = 100
category_names = ['person', 'dog', 'cat', 'car', 'bicycle', 'motorbike', 'traffic light', 'bird', 'truck']  # Classes que iremos baixar
data_dir = '/content/coco_yolo'
train_dir = os.path.join(data_dir, 'images/train')
labels_dir = os.path.join(data_dir, 'labels/train')
img_width = 150 # tamanho da imagem utilizada pelo yolo

# Cria as pastas
os.makedirs(train_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

def convert_bbox_to_yolo(bbox, img_width, img_height):
    """Converte bounding box do formato COCO para YOLO."""
    x, y, width, height = bbox
    x_center = ((x + (x + width)) / 2) / img_width # Correção: Calcula o centroide corretamente
    y_center = ((y + (y + height)) / 2) / img_height
    width_norm = width / img_width
    height_norm = height / img_height
    return x_center, y_center, width_norm, height_norm

def loadYoloDataset():
    """Função para baixar e salvar imagens com bounding boxes no formato YOLO."""

    # Itera sobre as imagens
    for image_data in images_data[:num_images_to_download]: #limitar o número de imagens
        img_id = image_data['id']
        file_name = image_data['file_name']
        img_width = image_data['width']
        img_height = image_data['height']

        # Construir o arquivo de anotação (YOLO)
        annotation_file = os.path.join(labels_dir, file_name.replace('.jpg', '.txt').replace('.png', '.txt'))

        # Baixar as imagens do site
        image_url = image_data['coco_url']
        try:
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            img.save(os.path.join(train_dir, file_name))

            annotation_content = []
            for annotation in coco_data['annotations']:
                if annotation['image_id'] == img_id and annotation['category_id'] in category_ids:
                    category_name = next((cat['name'] for cat in coco_data['categories'] if cat['id'] == annotation['category_id']), None) #Busca o nome da categoria pelo id
                    if category_name:
                        category_index = category_names.index(category_name)
                        bbox = annotation['bbox']
                        x_center, y_center, width_norm, height_norm = convert_bbox_to_yolo(bbox, img_width, img_height)
                        annotation_content.append(f"{category_index} {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}")

            # Verifica se gerou o conteudo
            if annotation_content:
                print(f"Anotações geradas para {file_name}: {annotation_content}")
                # Escreve no arquivo
                with open(annotation_file, 'w') as f:
                    f.write('\n'.join(annotation_content))
                print(f"Arquivo de anotação {annotation_file} criado.")
            else:
                print(f"Nenhuma anotação encontrada para {file_name}")

        except Exception as e:
            print(f"Erro ao processar a imagem {file_name}: {e}")

    print("Imagens e anotações baixadas e convertidas com sucesso!")

# 5. Chamada da função para criar os arquivos
# Baixa as anotações
annotations_url = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
annotations_zip_path = os.path.join(data_dir, 'annotations_trainval2017.zip')
annotations_dir = os.path.join(data_dir, 'annotations')
if not os.path.exists(annotations_zip_path):
    print("Baixando anotações...")
    !wget -q "{annotations_url}" -O "{annotations_zip_path}"
    print("Extraindo anotações...")
    !unzip -q "{annotations_zip_path}" -d "{data_dir}"

instances_path = os.path.join(annotations_dir, 'instances_train2017.json')
with open(instances_path, 'r') as f:
    coco_data = json.load(f)

# 1. Mapear IDs de imagens para IDs de categorias
image_to_categories = {}
for ann in coco_data['annotations']:
    image_id = ann['image_id']
    category_id = ann['category_id']
    if image_id not in image_to_categories:
        image_to_categories[image_id] = set() # Usando um set para ter elementos únicos
    image_to_categories[image_id].add(category_id)

# 2. Filtrar IDs de imagens com as categorias desejadas
category_ids = [cat['id'] for cat in coco_data['categories'] if cat['name'] in category_names]
images_data = [img for img in coco_data['images'] if img['id'] in image_to_categories and any(cat_id in image_to_categories[img['id']] for cat_id in category_ids)]

# 5. Chamada da função para criar os arquivos
loadYoloDataset()

# 1. Clonar o repositório YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clonar
%cd yolov5

# 2. Instalar as dependências
!pip install -r requirements.txt  # instalar dependências

# 3. Criar o arquivo de configuração YAML para o dataset
import yaml

data = {
    'train': '/content/coco_yolo/images/train',
    'val': '/content/coco_yolo/images/train',  # Use os mesmos dados para validação (para simplificar)
    'nc': len(category_names),  # Número de classes
    'names': category_names  # Nomes das classes
}

with open('coco.yaml', 'w') as f:
    yaml.dump(data, f)

# Imprimi o conteudo do arquivo.
print("\nConteúdo do arquivo YAML:")
!cat coco.yaml

# 1. Definir o modelo pre-treinado
pretrained_weights = 'yolov5s.pt' # Usa o modelo pequeno (mais rápido)

# 2. Definir o tamanho do batch, e as épocas
epochs = 20  #Aumentei o número de épocas
batch_size = 16

# 3. Treinar!
!python train.py --img {img_width} --batch {batch_size} --epochs {epochs} --data coco.yaml --weights {pretrained_weights} --cache

# 4. Testar
#test_image = '/content/drive/MyDrive/teste/meu_teste.jpg' #imagem do google drive
test_image = '/content/yolov5/data/images/bus.jpg' #imagem de exemplo do yolo

!python detect.py --weights runs/train/exp/weights/best.pt --img {img_width} --conf 0.5 --source "{test_image}"

Mounted at /content/drive
Baixando anotações...
Extraindo anotações...
Anotações geradas para 000000391895.jpg: ['0 0.651281 0.479236 0.240437 0.835361', '0 0.765000 0.546861 0.056125 0.133611', '4 0.783320 0.557778 0.047859 0.097167']
Arquivo de anotação /content/coco_yolo/labels/train/000000391895.txt criado.
Anotações geradas para 000000522418.jpg: ['0 0.798250 0.494073 0.401250 0.988146']
Arquivo de anotação /content/coco_yolo/labels/train/000000522418.txt criado.
Anotações geradas para 000000184613.jpg: ['0 0.453790 0.562917 0.321660 0.734821', '0 0.091910 0.389018 0.150660 0.437202', '0 0.136450 0.330908 0.091940 0.373661', '0 0.085700 0.208482 0.088560 0.149881', '0 0.037660 0.384896 0.072320 0.313899', '0 0.705520 0.233140 0.037920 0.085506', '0 0.741540 0.234821 0.034880 0.073750', '0 0.778340 0.229345 0.025640 0.096845', '0 0.839140 0.222054 0.023840 0.080952', '0 0.642370 0.217128 0.027380 0.094435', '0 0.585840 0.214152 0.019520 0.088423', '0 0.551810 0.213839 0.012740 0.09